In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# ##Define Graphing Function

# In this section, we define the function make_graph. You don't have to know how the function works, you should only care
# about the inputs. It takes a dataframe with stock data (dataframe must contain Date and Close columns), a dataframe with
# revenue data (dataframe must contain Date and Revenue columns), and the name of the stock.




In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [6]:
# Question 1: Use yfinance to Extract Stock Data

# Create a Ticker object for Tesla (TSLA)
tesla = yf.Ticker("TSLA")

# Fetch historical stock data for Tesla, covering the entire available period
tesla_data = tesla.history()

# Reset the index of the DataFrame
tesla_data.reset_index(inplace=True)

# Display the first few rows of the fetched stock data
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-10-04 00:00:00-04:00,248.139999,261.859985,247.600006,261.160004,129721600,0.0,0.0
1,2023-10-05 00:00:00-04:00,260.000000,263.600006,256.250000,260.049988,119159200,0.0,0.0
2,2023-10-06 00:00:00-04:00,253.979996,261.649994,250.649994,260.529999,117947000,0.0,0.0
3,2023-10-09 00:00:00-04:00,255.309998,261.359985,252.050003,259.670013,101377900,0.0,0.0
4,2023-10-10 00:00:00-04:00,257.750000,268.940002,257.649994,263.619995,122656000,0.0,0.0


In [7]:
# # Question 2: Use Webscraping to Extract Tesla Revenue Data
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find("table", class_="historical_data_table")

    if table:
        rows = table.find_all("tr")

        data = []
        for row in rows[1:]:  # Skipping the header row
            cells = row.find_all(["th", "td"])
            date = cells[0].get_text(strip=True)
            tesla_revenue = cells[1].get_text(strip=True)

            # Removing non-numeric characters from the revenue string and converting to float
            revenue_value = float(tesla_revenue.replace('$', '').replace(',', ''))
            data.append({'Date': date, 'Revenue': revenue_value})

        # Creating a DataFrame from the extracted data
        df = pd.DataFrame(data)

        # Displaying the last 5 rows using the tail function
        tesla_revenue_last_5_rows = df.tail(5)
        print(tesla_revenue_last_5_rows)
    else:
        print("Table not found on the page.")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


    Date  Revenue
9   2013   2013.0
10  2012    413.0
11  2011    204.0
12  2010    117.0
13  2009    112.0


In [10]:
# Question 3: Use yfinance to Extract Stock Data
gamestop = yf.Ticker("GME")
gme_data=gamestop.history()
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-10-04 00:00:00-04:00,14.77,14.940000,14.44,14.87,2586400,0.0,0.0
1,2023-10-05 00:00:00-04:00,14.70,14.980000,14.58,14.71,1945100,0.0,0.0
2,2023-10-06 00:00:00-04:00,14.59,15.260000,14.58,15.08,2458300,0.0,0.0
3,2023-10-09 00:00:00-04:00,14.77,15.500000,14.72,15.40,2276300,0.0,0.0
4,2023-10-10 00:00:00-04:00,15.50,16.190001,15.40,16.00,2915900,0.0,0.0


In [11]:
#Question 4: Use Webscraping to Extract GME Revenue Data

url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find("table", class_="historical_data_table")

    if table:
        rows = table.find_all("tr")

        data = []
        for row in rows[1:]:  # Skipping the header row
            cells = row.find_all(["th", "td"])
            date = cells[0].get_text(strip=True)
            gme_revenue = cells[1].get_text(strip=True)
            
            # Removing non-numeric characters from the revenue string and converting to float
            revenue_value = float(gme_revenue.replace('$', '').replace(',', ''))
            data.append({'Date': date, 'Revenue': revenue_value})

        # Creating a DataFrame from the extracted data
        df = pd.DataFrame(data)

        # Displaying the last 5 rows using the tail function
        gme_revenue_last_5_rows = df.tail(5)
        print(gme_revenue_last_5_rows)
    else:
        print("Table not found on the page.")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


    Date  Revenue
10  2013   8887.0
11  2012   9551.0
12  2011   9474.0
13  2010   9078.0
14  2009   8806.0


In [12]:
make_graph(tesla_data, tesla_revenue, 'Tesla Stock Data Graph')

C:\Users\luis\AppData\Local\Temp\ipykernel_7680\1276540637.py:3: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



AttributeError: 'str' object has no attribute 'Date'

In [13]:
make_graph(gme_data, gme_revenue, 'GameStop Stock Data Graph')

C:\Users\luis\AppData\Local\Temp\ipykernel_7680\1276540637.py:3: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



AttributeError: 'str' object has no attribute 'Date'